In [ ]:
import os
os.chdir("..")
import json

import pandas as pd
import ast
import tabulate

from tool.gender_checker import get_personal_titles
from tool.annotations_utils import read_annotations, has_intersection, fix_personal_titles, personal_titles_stats
from tool.ner_metrics import *
from tool.file_and_directory_management import read_file_to_list

In [ ]:
titles = read_file_to_list('data/novels_titles/combined_set.txt')

In [ ]:
gold_standard_path = "data/testing_sets/test_person_gold_standard_corrected"

In [ ]:
results = []
for title in titles:
    annotations_person, _ = data_from_json(os.path.join(gold_standard_path, title + '.json'))

    prediction_path = "experiments/protagonist/ner"
    predictions_person, _ = data_from_json(os.path.join(prediction_path, title + '.json'))

    gold_standard_path = "data/testing_sets/test_names_gold_standard_corrected"
    annotations_names, _ = data_from_json(os.path.join(gold_standard_path, title+ '.json'))

    prediction_path = "experiments/protagonist"
    predictions_names, _ = data_from_json(os.path.join(prediction_path, title+ '.json'))

    missing = 0
    incorrect = 0
    protagonist_person = 0 
    protagonist_protagonist = 0
    person_protagonist = 0
    for sent_id, (sent_gold_entities, sent_matcher_entities_person, sent_matcher_entities) in enumerate(
            zip(annotations_names, predictions_person, predictions_names)):
        sent_matcher_entities_person = [x[:3] for x in sent_matcher_entities_person]
        algorithm = 0

        for gold_entity in sent_gold_entities:
            if gold_entity in sent_matcher_entities:  # if there is the same entity in predictions -> TP
                pass
            elif gold_entity[:2] + ["PERSON"] in sent_matcher_entities_person:
                pred_entity = [x for x in sent_matcher_entities if x[0] == gold_entity[0] and x[1] == gold_entity[1]]
                assert len(pred_entity) == 1
                algorithm += 1
                if gold_entity[2] == "PERSON":
                    person_protagonist += 1
                elif pred_entity[0][2] == "PERSON":
                    protagonist_person += 1
                else:
                    protagonist_protagonist += 1
            else:
                missing += 1
        for matcher_entity in sent_matcher_entities:
            if matcher_entity not in sent_gold_entities:
                incorrect += 1
        incorrect -= algorithm    
    title_results = {'Title': title.split('.')[0].replace('_', ' '), 
                 '# missing person errors': missing, 
                 '# incorrect person errors': incorrect, 
                 '# protagonist-person errors': protagonist_person,
                 '# person-protagonist errors': person_protagonist, 
                 '# protagonist-protagonist errors':  protagonist_protagonist}
    results.append(title_results)
df = pd.DataFrame(results)

In [ ]:
print(tabulate.tabulate(df, headers = df.columns, tablefmt='latex_booktabs'))